In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')

In [2]:
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from src.Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Utils.utils import create_URM,create_ICM,combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from skopt.space import Real, Integer, Categorical

In [3]:
URM = create_URM()
ICM = create_ICM()


URM_train_validation,URM_test = split_train_in_two_percentage_global_sample(URM,train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation,train_percentage=0.85)

combined_matrices = combine_matrices(ICM=ICM, URM=URM_train)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13638 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13632 ( 0.1%) Users that have less than 1 test interactions


In [ ]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

hyperparameters_range_dictionary = {
    "topK":Integer(10,600),
    "l2_norm":Real(0.00001,0.01)
}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [combined_matrices],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

tuning_class = SearchBayesianSkopt(recommender_class=EASE_R_Recommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases*0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    recommender_input_args_last_test=recommender_input_args_last_test,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize= 10,
                    n_cases= n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=EASE_R_Recommender.RECOMMENDER_NAME,
                    save_model="best"
                    )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 314, 'l2_norm': 0.003558484214357064}
EASE_R_Recommender: Fitting model... 
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 18059 (100.0%), 374.76 column/sec. Elapsed time 48.19 sec
EASE_R_Recommender: Fitting model... done in 12.89 min
EvaluatorHoldout: Processed 13632 (100.0%) in 31.72 sec. Users per second: 430
SearchBayesianSkopt: New best config found. Config 0: {'topK': 314, 'l2_norm': 0.003558484214357064} - results: PRECISION: 0.0517532, PRECISION_RECALL_MIN_DEN: 0.0519520, RECALL: 0.0110433, MAP: 0.0192212, MAP_MIN_DEN: 0.0192897, MRR: 0.1416061, NDCG: 0.0541544, F1: 0.0182025, HIT_RATE: 0.3865904, ARHR_ALL_HITS: 0.1645208, NOVELTY: 0.0059937, AVERAGE_POPULARITY: 0.3529397, DIVERSITY_MEAN_INTER_LIST: 0.9812953, DIVERSITY_HERFINDAHL: 0.9981223, COVERAGE_ITEM: 0.1672296, COVERAGE_ITEM_CORRECT: 0.0431918, COVERAGE_USER: 0.9986813, COVERAGE_US